# Minishell Export/Unset Debug Analysis

## Critical Issues Identified

Based on the test results, we have several critical issues in the minishell:

### 🚨 **Memory Corruption Issues**
- `free(): double free detected in tcache 2`
- `double free or corruption (fasttop)`
- Exit code 134 (SIGABRT) indicating program crashes

### 🚨 **Command Parsing Issues**
- Commands being split incorrectly: `echo` → `"Command 'it' not found"`
- Garbage characters in commands: `"Command '�@�[' not found"`
- Commands not executing at all: Basic commands fail

### 🚨 **Export/Unset Functionality Issues**
- Export multiple variables fails
- Invalid identifier validation crashes the program
- Variable expansion produces corrupted output
- Memory management in export/unset functions

In [ ]:
import os
import subprocess
import re
import tempfile
from pathlib import Path

# Set up the minishell path
MINISHELL_PATH = "/mnt/c/Users/user/Desktop/Minishell"
MINISHELL_BINARY = f"{MINISHELL_PATH}/minishell"

# Create test output directory
os.makedirs("/tmp/minishell_debug", exist_ok=True)

print("🔧 Environment Setup Complete")
print(f"Minishell path: {MINISHELL_PATH}")
print(f"Binary: {MINISHELL_BINARY}")
print(f"Working directory: {os.getcwd()}")

## 🧠 Memory Management Analysis

### Root Cause Analysis

The **"double free detected"** errors indicate that the same memory location is being freed multiple times. This is a critical bug that causes immediate program termination.

### Key Memory Issues Found:

1. **Export Function Memory Flow**:
   ```c
   // In handle_export_var():
   parse_arg_var(cmd->arg[i], &name, &value);  // Allocates name and value
   // ... processing ...
   free_name_value(name, value);               // Frees name and value
   ```

2. **Potential Double-Free Sources**:
   - `parse_arg_var()` allocates memory with `ft_strdup()`
   - `create_var()` also duplicates the name/value with `ft_strdup()`
   - Both original and duplicated memory might be freed

3. **Variable Creation Process**:
   ```c
   new_var = create_var(name, value, true);    // Duplicates name/value internally
   free_name_value(name, value);               // Frees original - CORRECT
   // But if create_var fails, we might have issues
   ```

In [ ]:
def run_minishell_command(command, timeout=5):
    """Run a minishell command and capture output, return code, and any crashes"""
    try:
        # Create a temporary script file
        with tempfile.NamedTemporaryFile(mode='w', suffix='.sh', delete=False) as f:
            f.write(f'echo "{command}" | timeout {timeout}s {MINISHELL_BINARY}\n')
            script_path = f.name
        
        # Run the command
        result = subprocess.run(['bash', script_path], 
                              capture_output=True, 
                              text=True, 
                              timeout=timeout+2)
        
        # Clean up
        os.unlink(script_path)
        
        return {
            'stdout': result.stdout,
            'stderr': result.stderr,
            'returncode': result.returncode,
            'crashed': result.returncode == 134,  # SIGABRT
            'timeout': result.returncode == 124   # timeout exit code
        }
    except subprocess.TimeoutExpired:
        return {
            'stdout': '',
            'stderr': 'Process timed out',
            'returncode': -1,
            'crashed': False,
            'timeout': True
        }
    except Exception as e:
        return {
            'stdout': '',
            'stderr': str(e),
            'returncode': -1,
            'crashed': False,
            'timeout': False
        }

# Test basic functionality
print("🧪 Testing basic minishell functionality:")
basic_result = run_minishell_command("exit")
print(f"Basic exit test: Return code {basic_result['returncode']}, Crashed: {basic_result['crashed']}")

# Test a simple export
export_result = run_minishell_command("export test=hello")
print(f"Basic export test: Return code {export_result['returncode']}, Crashed: {export_result['crashed']}")
if export_result['stderr']:
    print(f"Export stderr: {export_result['stderr'][:200]}...")

## 🔧 Critical Memory Fixes Applied

### **Fix 1: Double-Free in `name_invalid()` Function**

**Problem**: `name_invalid()` was freeing memory that would be freed again by the caller.

```c
// OLD CODE (BUGGY):
void name_invalid(char *name, char *value)
{
    print_export_error(name);
    free(name);  // ❌ Double-free when caller also calls free_name_value()
    free(value); // ❌ Double-free
}

// NEW CODE (FIXED):
void name_invalid(char *name, char *value)
{
    print_export_error(name);
    // ✅ Don't free here - caller handles cleanup
    (void)value; // Suppress unused parameter warning
}
```

### **Fix 2: Memory Leak in `create_var()` Function**

**Problem**: When `init_var_fields()` failed, allocated memory wasn't freed.

```c
// OLD CODE (BUGGY):
t_env_var *create_var(char *name, char *value, bool booling)
{
    // ... allocation code ...
    if (init_var_fields(new_var, name, value, booling))
        return (new_var);
    return (NULL); // ❌ Memory leak - new_var not freed
}

// NEW CODE (FIXED):
t_env_var *create_var(char *name, char *value, bool booling)
{
    // ... allocation code ...
    if (init_var_fields(new_var, name, value, booling))
        return (new_var);
    free(new_var); // ✅ Free allocated memory on failure
    return (NULL);
}
```

### **Fix 3: Missing Memory Cleanup in `handle_export_var_cd()`**

**Problem**: Function didn't free the allocated name/value parameters.

```c
// FIXED: Added proper cleanup in all exit paths
int handle_export_var_cd(char *name, char *value, t_shell *shell, int status)
{
    // ... validation and processing ...
    
    // ✅ Always free the input parameters before returning
    free(name);
    free(value);
    return (status);
}
```

## 🔍 Command Parsing Issues Analysis

### **Garbage Characters in Commands**

The test output shows commands being parsed incorrectly:
- `echo` becomes `"Command 'it' not found"`
- Commands with garbage characters: `"Command '�@�[' not found"`

### **Root Causes Identified**

1. **Buffer Overflows in Variable Expansion**: The `expand_variables_in_token()` function may be corrupting memory during string concatenation.

2. **Improper Memory Management in `ft_strjoin_free()`**: This function frees both input strings, which can cause issues if one of them is still needed.

3. **Environment Variable Corruption**: The environment array updating might be corrupting the command parsing buffers.

### **Additional Fixes Needed**

#### **1. Safer String Joining Function**
```c
// Current problematic implementation:
char *ft_strjoin_free(char *s1, char *s2)
{
    char *result = ft_strjoin(s1, s2);
    free(s1);  // ❌ Always frees both - can cause issues
    free(s2);  // ❌ if caller doesn't expect this
    return (result);
}

// Suggested safer approach:
char *ft_strjoin_free_first(char *s1, char *s2)
{
    char *result = ft_strjoin(s1, s2);
    free(s1);  // Only free the first string
    return (result);
}
```

#### **2. Bounds Checking in Variable Expansion**
- Add length validation in environment variable substitution
- Ensure null termination of all string operations
- Validate memory allocation results before use

## 🎯 Complete Solution Summary

### **Immediate Actions to Take**

1. **Recompile minishell** with the memory fixes:
   ```bash
   cd /mnt/c/Users/user/Desktop/Minishell
   make clean && make
   ```

2. **Test the basic export functionality**:
   ```bash
   echo "export test=hello" | ./minishell
   echo "export invalid_1x" | ./minishell  # Should not crash
   ```

### **Files Modified**

- ✅ **`execute/export_utils_2.c`**: Fixed memory leak in `create_var()`
- ✅ **`execute/export_utils.c`**: Fixed double-free in `name_invalid()`
- ✅ **`execute/exec_buildins_2.c`**: Added proper cleanup in `handle_export_var_cd()`

### **Expected Results After Fixes**

After applying these fixes, the following should work:

✅ **Export multiple variables**: `export x1="abcd" x2="123" x3="shoaib"`
✅ **Invalid identifier handling**: `export 1x` (should return error without crashing)
✅ **Mixed identifiers**: `export test="123" 1test="test" test3="abcd"`
✅ **Export without assignment**: `export test1`
✅ **Export empty value**: `export test2=`
✅ **Variable expansion**: `export test=$HOME`

### **Testing Commands**

Run these to verify the fixes:
```bash
# Test 1: Basic export
echo 'export test=hello' | ./minishell

# Test 2: Invalid identifier (should not crash)
echo 'export 1x' | ./minishell

# Test 3: Multiple variables
echo 'export x1="abcd" x2="123" x3="shoaib"' | ./minishell
```

### **Remaining Issues to Monitor**

1. **Command Parsing Corruption**: If garbage characters still appear, investigate:
   - Variable expansion buffer overflows
   - Environment array corruption
   - Parser memory management

2. **Variable Expansion Issues**: Monitor for:
   - Incorrect variable substitution
   - Memory corruption during string concatenation
   - Quote handling problems

The memory management fixes should resolve the immediate crashes and double-free errors, allowing the export/unset functionality to work properly.